In [7]:
#import modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_blobs
from sklearn import metrics
from pandas_profiling import ProfileReport

%matplotlib inline
plt.style.use('ggplot')

In [11]:
file = pd.read_csv('data/whisky.csv')

In [8]:
profile = ProfileReport(file, title='Pandas Profiling Report', html={'style':{'full_width':True}})

build report structure: 100%|██████████| 1/1 [00:26<00:00, 26.56s/it]


In [10]:
profile.to_widgets()